In [50]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import os
import tweepy
import pandas as pd
import json


# Add your Twitter API credentials
consumer_key = "xm94pAUphNozJUwY4SzFM8nnT"
consumer_secret = "k7gFj8K4kE344d8kyBA67xabwL5XMiZXGmDmTjnx1PDWuI0bRO"
access_key = "4512685235-ckOFfc3r6QiASVdJBYNUih1CPZWHpVOquSDSon3"
access_secret = "AAEJmUPJtQTYz36qfjjmmZx0fyqH6cHaXjuyM0KwBDMHf"

In [51]:
# Handling authentication with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# Create a wrapper for the API provided by Twitter
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [52]:
search_words = "#xenophobia"
date_since = "2017-05-18"

# Define until what date we are looking for tweets
date_until = ""

# Total tweets to gather in our search
totalTweets = 100000

# Numbers of tweets to return per page, max is 100. Default is 15.
count = 100

# Filter by language
lang = "en"

'''Filter by latitude,longitude,radius.
# 37.781157 -122.398720 1mi'''
geocode = ""

# Filter by recent, popular or mixed.
result_type = "recent"

'''Include info on entities found in Tweets, including hashtags,
links, and mentions. Set to True or False'''
include_entities = True

# Set the name for CSV file  where the tweets will be saved
filename = "xenophobia_tweets"
new_search = search_words + " -filter:retweets"


In [53]:
date_since = "2017-05-18"

# Define until what date we are looking for tweets
date_until = ""

# Total tweets to gather in our search
totalTweets = 100000

# Numbers of tweets to return per page, max is 100. Default is 15.
count = 100

# Filter by language
lang = "en"

'''Filter by latitude,longitude,radius.
# 37.781157 -122.398720 1mi'''
geocode = ""

# Filter by recent, popular or mixed.
result_type = "recent"

'''Include info on entities found in Tweets, including hashtags,
links, and mentions. Set to True or False'''
include_entities = True

# Set the name for CSV file  where the tweets will be saved
filename = "xenophobia_tweets"
new_search = search_words + " -filter:retweets"


In [54]:
# Function for handling pagination in our search
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            print('Reached rate limite. Sleeping for >15 minutes')
            time.sleep(15 * 61)



In [55]:
def search_tweets(new_search, date_since):
    tweets = limit_handled(tweepy.Cursor(api.search, 
                          q=search_words,
                           lang="en",
                           since=date_since).items(totalTweets))
   

        
    twitter_object = [[
               tweet.text, tweet.retweet_count, tweet.favorited, tweet.truncated, tweet.id_str, tweet.in_reply_to_screen_name,
               tweet.source, tweet.retweeted, tweet.created_at, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id, 
               tweet.lang, tweet.user.listed_count, tweet.user.verified, tweet.user.location, tweet.user.statuses_count,
               tweet.user.followers_count, tweet.user.favourites_count, tweet.user.protected, tweet.user.time_zone,
               tweet.user.url] for tweet in tweets]
    try:

        tweet_text = pd.DataFrame(data=twitter_object, columns=["text","retweet_count","favorited","truncated","id_str",
                                                        "in_reply_to_screen_name","source","retweeted","created_at","in_reply_to_status_id_str","in_reply_to_user_id",
                                                       "lang","listed_count","verified","location","statuses_count","followers_count","favourites_count","protected","time_zone","user_url"])

        #writes data frame to a csv file
        tweet_text.to_csv('tweets_sample.csv')

    except Exception as e:
            print('Encountered Exception:', e)
            pass

In [56]:
# save twitter data to a dataframe and csv file

# def save_to_df_csv(twitter_data):
    
# #save twitter data as a dataframe    
#     tweet_text = pd.DataFrame(data=twitter_data, columns=["text","retweet_count","favorited","truncated","id_str",
#                                                     "in_reply_to_screen_name","source","retweeted","created_at","in_reply_to_status_id_str","in_reply_to_user_id",
#                                                    "lang","listed_count","verified","location","statuses_count","followers_count","favourites_count","protected","time_zone","user_url"])
    
#     #writes data frame to a csv file
#     tweet_text.to_csv('tweets_sample.csv')
    

In [57]:
# tweet_text.to_csv('tweets_sample.csv')

In [58]:
def work():
    
        # Initializing the Twitter search
        try:
            data = search_tweets(search_words, date_since)

        # Stop temporarily when hitting Twitter rate Limit
        except tweepy.RateLimitError:
            print("RateLimitError...waiting ~15 minutes to continue")
            time.sleep(1001)
            search_tweets(search_words, date_since)

        # Stop temporarily when getting a timeout or connection error
        except (Timeout, ssl.SSLError, ReadTimeoutError,
                ConnectionError) as exc:
            print("Timeout/connection error...waiting ~15 minutes to continue")
            time.sleep(1001)
            search_tweets(search_words, date_since)

        # Stop temporarily when getting other errors
        except tweepy.TweepError as e:
            if 'Failed to send request:' in e.reason:
                print("Time out error caught.")
                time.sleep(1001)
                search_tweets(search_words, date_since)
            elif'Too Many Requests' in e.reason:
                print("Too many requests, sleeping for 15 min")
                time.sleep(1001)
                search_tweets(search_words, date_since)
            else:
                print(e)
                print("Other error with this user...passing")
                pass


   


if __name__ == '__main__':

    work()

NameError: name 'Timeout' is not defined